# Stochastic Volume Calculations

Knowing the volume of different regions in a model can be important for tally normalization, which we will illustrate in a later tutorial. In some cases, you can obtain the volume of a cell analytically, but this is not always possible for more complex regions. For these regions, we can use OpenMC's stochastic volume calculation mode.

In [1]:
import openmc

We will load a pre-built example in OpenMC to save us a lot of the effort in model building.

In [2]:
model = openmc.examples.pwr_core()

In [ ]:
model.geometry.root_universe.plot(width=(500, 500), pixels=(500, 500))

In [ ]:
model.geometry.root_universe.plot(width=(22, 22), origin=(0, 0, 0), pixels=(500, 500))

This model doesn't contain any particularly complex geometry, but there are still regions for which it would be onerous to compute volumes analytically, such as the region between the baffle and barrel (dark green).

In [ ]:
model.geometry.root_universe.plot(width=(100, 100), origin=(130, 130, 0), pixels=(500, 500))

Volumes can be computed using a few different domain types: materials, cells, and universes. The number of atoms for different nuclides can be computed as well based on the volumes of materials and their various atom percentages.

Volume calculations are performed by randomly selecting locations in a specified bounding box. The number of locations for where a domain item (cell, material, universe) is found indicates how much of the bounding box is occupied by this item.

$$ V_{i} = \frac{N_{i}}{N} V_{box}$$

This straightforward method is a relative to one of the most well-known uses of Monte Carlo, [the computation of $\pi$](https://blogs.sas.com/content/iml/2016/03/14/monte-carlo-estimates-of-pi.html#:~:text=To%20compute%20Monte%20Carlo%20estimates,the%20curve%20is%20%CF%80%20%2F%204.).

Let's set up a volume calculation using the materials from this PWR core model. We need to define the bounding box for the volume calculation, which we take to encompass the entire domain.

In [3]:
lower_left = (-250, -250, -250)
upper_right = (250, 250, 250)

Let's compute the volumes of the different materials in the domain.

In [4]:
materials = model.materials

print(len(materials))
print(materials[:2])

12
[Material
	ID             =	1
	Name           =	UOX fuel
	Temperature    =	None
	Density        =	10.062 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	True
	S(a,b) Tables  
	Nuclides       
	U234           =	4.9476e-06   [ao]
	U235           =	0.00048218   [ao]
	U238           =	0.021504     [ao]
	Xe135          =	1.0801e-08   [ao]
	O16            =	0.045737     [ao]
, Material
	ID             =	2
	Name           =	Zircaloy
	Temperature    =	None
	Density        =	5.77 [g/cm3]
	Volume         =	None [cm^3]
	Depletable     =	False
	S(a,b) Tables  
	Nuclides       
	Zr90           =	0.5145       [ao]
	Zr91           =	0.1122       [ao]
	Zr92           =	0.1715       [ao]
	Zr94           =	0.1738       [ao]
	Zr96           =	0.028        [ao]
]


In [6]:
vc = openmc.VolumeCalculation(materials, samples=10_000_000, lower_left=lower_left, upper_right=upper_right)

Now we apply the volume calculation to the model's settings and we're ready to run!

In [7]:
model.settings.volume_calculations = [vc]

In [8]:
model.calculate_volumes()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

We can see that the volume information was printed to the screen after the solve completes. We can also access this information from an HDF5 file which OpenMC writes. We can directly apply this volume information to the materials in our model, if we want to use it later on for tally normalization.

In [9]:
vc.load_results('volume_1.h5')

In [10]:
model.geometry.add_volume_information(vc)

In [11]:
model.materials[0].volume

12296725.0

Pretty easy overall! Now we'll look use a different volume calculation to compute volumes according to the different cells in this PWR. In this case, we will set `domains` to all of the cells in the problem.

In [18]:
cells = list(model.geometry.get_all_cells().values())

cell_vc = openmc.VolumeCalculation(domains=cells, samples=10_000_000, lower_left=lower_left, upper_right=upper_right)
model.settings.volume_calculations = [cell_vc]

In [19]:
model.calculate_volumes()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

In [16]:
cells[0].volume

25114225.0

We can also view the number of atoms of each nuclide in each cell, through a Pandas dataframe.

In [20]:
cell_vc.atoms_dataframe

,Cell,Nuclide,Atoms
0,1,U234,(3.084+/-0.004)e+25
1,1,U235,(3.006+/-0.004)e+27
2,1,U238,(1.3406+/-0.0019)e+29
3,1,Xe135,(6.734+/-0.009)e+22
4,1,O16,(6.956+/-0.005)e+29
...,...,...,...
214,9,Fe57,(3.752+/-0.004)e+27
215,9,Fe58,(5.001+/-0.005)e+26
216,9,Ni58,(1.4961+/-0.0015)e+28
217,9,Mn55,(5.218+/-0.005)e+27


As we see in the simulation output, there is uncertainty associated with these results. The OpenMC volume calculation is *stochastic*, and is essentially obtained by "throwing darts" at the problem. The more samples used, the lower the statistical uncertainty.

All OpenMC tallies support triggers, which will continue increasing the number of samples taken until reaching some desired statistical uncertainty. We will also use these features for other tally types in a later tutorial.

In [21]:
vc.samples = 100_000
vc.set_trigger(15_000, 'std_dev')

In [22]:
model.settings.volume_calculations = [vc]

In [23]:
model.calculate_volumes()

                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################